# CSCI 209 Final Project
## Turned-Based Combat System

In [1]:
import json
with open("Moves.json", "r") as f:
    MOVESET = json.load(f)

with open("Types.json", "r") as f:
    TYPES = json.load(f)

import random

In [2]:
# CHANGE MOVESET HERE
moveset = {
    1: MOVESET["Small Heal"],
    2: MOVESET["Attack"],
    3: None,
    4: None
}

In [3]:
def start_game():
    first_actor = random.choice([0, 1])
    player_type = TYPES[str(random.choice([0, 2]))]
    enemy_type = TYPES[str(random.choice([0, 2]))]


    data = {
        "is_player_first": first_actor,
        "player_hp": player_type['base_hp'], 
        "player_type": player_type,
        "player_move_list": [],
        "enemy_hp": enemy_type['base_hp'],
        "enemy_type": enemy_type,
        "enemy_move_list": []
    }

    start_battle(data)

In [4]:
def start_battle(data):
    display_game_info(data)

    print("Your Battle Begins!\n")
    select_move(data)

def display_game_info(data):
    print("--PLAYER INFO--")
    print("Your Health:", data['player_hp'])
    print("Your Type:", data['player_type']['name'])
    print("\n--ENEMY INFO--")

    print("Enemy's Health:", data['enemy_hp'])
    print("Enemy's Type:", data['enemy_type']['name'],"\n")

    #print("Your Moveset:", print_moveset())

def print_moveset():
    for move in moveset.values:
        print(move)

In [5]:
# ENEMY AI

def do_enemy_action(data):
    # check if low on health
    e_hp = data['enemy_hp']
    e_type = data['enemy_type']

    if e_hp < (e_type['base_hp'] * e_type['heal_threshold']): # attempt to heal
        if random.random() < e_type['heal_probability']:
            data['enemy_hp'] += 10 # generic heal/replace w function?
            print("ENEMY used: HEAL")
            return
        else:
            data['player_hp'] -= 10 # generic attack/replace w function?
            print("ENEMY used: ATTACK")

            return
        
    data['player_hp'] -= 10
    print("ENEMY used: ATTACK")


            

    # check if best attack is greater than 
    pass

In [ ]:
def select_move(data):
    move_index_str = input("Select a move (type: 1-4):")
    # NEED TO ADD: validation
    move_index = int(move_index_str)

    move = moveset[move_index]
    execute_moves(move, data)

def execute_moves(move, data):

    if data['is_player_first'] == 1:
        pass
    else: 
        pass

    print('YOU used:', move['type'])
    if move['type'] == 'HEAL':
        data['player_hp'] += move['base_hp']
    elif move['type'] == 'ATTACK':
        data['enemy_hp'] -= move['base_hp']

    # NEED TO CHECK IF GAME OVER AFTER EACH ACTION
    do_enemy_action(data)


    hp = data['player_hp']
    enemy_hp = data['enemy_hp']

    print("- Your Health:", hp)
    print("- Enemy's Health:", enemy_hp)

    print("")

    check_game_over(data, hp, enemy_hp)
    

def check_game_over(data, hp, enemy_hp):
    if hp > 0 and enemy_hp > 0:
        select_move(data)
        return
    
    state = ""
    if hp > 0: state = "WON"
    else: state = "LOSE"
    game_end(state)

def game_end(result):
    if result == "WON":
        print("Congrats! You won!")
    elif result == "LOSE":
        print("You lose this time.")

In [7]:
start_game()

--PLAYER INFO--
Your Health: 70
Your Type: Fire

--ENEMY INFO--
Enemy's Health: 70
Enemy's Type: Fire 

Your Battle Begins!

YOU used: ATTACK
ENEMY used: ATTACK
- Your Health: 60
- Enemy's Health: 60

YOU used: ATTACK
ENEMY used: ATTACK
- Your Health: 50
- Enemy's Health: 50

YOU used: ATTACK
ENEMY used: ATTACK
- Your Health: 40
- Enemy's Health: 40

YOU used: ATTACK
ENEMY used: ATTACK
- Your Health: 30
- Enemy's Health: 30

YOU used: ATTACK
ENEMY used: ATTACK
- Your Health: 20
- Enemy's Health: 20

YOU used: ATTACK
ENEMY used: ATTACK
- Your Health: 10
- Enemy's Health: 10

YOU used: ATTACK
ENEMY used: HEAL
- Your Health: 10
- Enemy's Health: 10



KeyboardInterrupt: Interrupted by user